In [81]:
import numpy as np
import math as mat

In [82]:
from sympy import *

In [83]:
#READ ME!!!
#enter points here
#ONLY change whatever is written in this box do not change anything else!!!!!!!
# Refer to diagrams to understand assumptions and labeling
#After putting inputs run all code with shift+enter and once you reach the bottom cell you can read a nice output
g=9.81
mass=114.2 #kg mass on singular front wheel

Lower_Aft=Matrix([-200,30,161.2])*10**-3
Lower_Fore=Matrix([157,30,161.2])*10**-3
Lower_Kingpin=Matrix([75,349.5,161.2])*10**-3

Upper_Aft=Matrix([-23.089,33.6,539.1])*10**-3
Upper_Fore=Matrix([199.56,33.6,539.1])*10**-3
Upper_Kingpin=Matrix([35,285.5,560.1])*10**-3

Tierod_Kingpin=Matrix([-74.47,258.03,506.738])*10**-3
Tierod_Mount=Matrix([-74.47,144.78,506.738])*10**-3

Contact_Patch=Matrix([0,427.5+50,0])*10**-3

SA_Lowerarms=Matrix([60,240,400])*10**-3
SA_Chassis=Matrix([60,290,161.2])*10**-3

Force=Matrix([g*mass,g*mass,2*g*mass]) #condition set here
#with a 3 wheel car it (z-direction) should actually be 1g+0.5 g of what the rear wheel carries. So apart from race (ASC) regulatory calculations this is the correct calc:
#Force[2]=-(56.59*0.5+mass)*g

Lower_Aft


Matrix([
[  -0.2],
[  0.03],
[0.1612]])

In [84]:
#Normalizing all vectors as directions of the forces we'll be using a.k.a. directions of the arms
#Refer my notes on Slack for what c b d e f means 
Upper_Fore_Arm=Upper_Fore-Upper_Kingpin
Upper_Aft_Arm=Upper_Aft-Upper_Kingpin
Lower_Aft_Arm=Lower_Aft-Lower_Kingpin
Lower_Fore_Arm=Lower_Fore-Lower_Kingpin
Steering_Arm=Tierod_Mount-Tierod_Kingpin
SA_dir=(SA_Chassis-SA_Lowerarms)/((SA_Chassis-SA_Lowerarms).norm())
c=Lower_Fore_Arm/Lower_Fore_Arm.norm()
b=Lower_Aft_Arm/Lower_Aft_Arm.norm()
d=Upper_Aft_Arm/Upper_Aft_Arm.norm()
e=Upper_Fore_Arm/Upper_Fore_Arm.norm()
f=Steering_Arm/Steering_Arm.norm()
g=SA_dir
#Now setting up unknowns 
#the uknowns are the magnitudes of forces acting at the direction of the arms
#I say magnitudes but these are 2 force members so they might end up giving negative results which means the 2 force member is in compression or positive if it's in tension.
fb,fc,fd,fe,ff,fg=symbols('fb,fc,fd,fe,ff,fg')

#Setting up force vectors

Fb=fb*b
Fc=fc*c
Fd=fd*d
Fe=fe*e
Ff=ff*f
Fg=fg*g

#Knowns
Fa=Force


In [85]:
#Body is wheel + kingpin
#  moment origin point is contact patch
#I'll use this moment equation but can use any one of these for proving or sth else idk

#Moment arms
L=Upper_Kingpin-Contact_Patch
K=Lower_Kingpin-Contact_Patch
M=Tierod_Kingpin-Contact_Patch
LL=SA_Lowerarms-Contact_Patch

#Moments
#notice 2 arms act at the same point so you need to add their moments
MK=K.cross(Fb)+K.cross(Fc)#=0

ML=L.cross(Fd)+L.cross(Fe)

MM=M.cross(Ff)
MLL=LL.cross(Fg)




In [86]:
#Equations

Eq1=Eq(MK+MLL+ML+MM,Matrix([0,0,0])) #this used

#Also sum of all forces
Eq5=Eq(Fb+Fc+Fd+Fe+Ff,Fa)
#bottom all 3 direction, top x can't be a force cause it needs a moment around that axis
#that adjustment was made beforehand


sol=solve((Eq1,Eq5),(fb,fc,fd,fe,ff,fg))

sol

{fb: -6474.67113381086,
 fc: 26977.5511988591,
 fd: -9025.45005067686,
 fe: -21685.5238672632,
 ff: 4532.86468821161,
 fg: 64894.1821243875}

In [87]:
#If you want to use other moment origins for calculations where you see for example what would be the braking force if you have x amount of force on an arm you can use these equations.
# origin point is lower arm kingpin attachment

#Moment arms
Q=Contact_Patch-Lower_Kingpin
O=Upper_Kingpin-Lower_Kingpin
P=Tierod_Kingpin-Lower_Kingpin
QQ=SA_Lowerarms-Lower_Kingpin
#Moments
MQ=Q.cross(Fa)
MO=O.cross(Fd)+O.cross(Fe)
MP=P.cross(Ff)
MQQ=QQ.cross(Fg)

#origin point is upper arm kingpin attachment

#Moment arms

R=Tierod_Kingpin-Upper_Kingpin
S=Lower_Kingpin-Upper_Kingpin
T=Contact_Patch-Upper_Kingpin

#Moments

MR=R.cross(Ff)
MS=S.cross(Fb)+S.cross(Fc)
MT=T.cross(Fa)

#origin point is tierod kingpin attachment

#Moment arms
U=Upper_Kingpin-Tierod_Kingpin
V=Lower_Kingpin-Tierod_Kingpin
Y=Contact_Patch-Tierod_Kingpin

#Moments
MU=U.cross(Fd)+U.cross(Fe)
MV=V.cross(Fb)+V.cross(Fc)
MY=Y.cross(Ff)

Eq2=Eq(MQ+MO+MP,Matrix([0,0,0]))
Eq3=Eq(MR+MS+MT,Matrix([0,0,0]))
Eq4=Eq(MU+MV+MY,Matrix([0,0,0]))

In [88]:
#THIS DIDN'T WORK :(
#Calculations regarding the shock absorber 
#Shock absorber as rigid body, two force member
#Body is lower control arms, refer to diagrams
#
#Shock absorber direction
#SA_dir=(SA_Lowerarms-SA_Chassis)/((SA_Lowerarms-SA_Chassis).norm())
#
#Shock absorber unknown force magnitude, force vector, acc -> force that comes from a bump that moves the shock absorber 
#fg=symbols('fg')
#lower_kingpin_reaction=symbols('lower_kingpin_reaction')
#Fg=fg*SA_dir
#lkrx,lkry,lkrz=symbols('lkrx,lkry,lkrz',real=True) #Lower_kingpin_reaction directions
#LKR=Matrix([lkrx,lkry,lkrz])#Lower_Kingpin_reaction direction vector
#LKR=LKR/LKR.norm() #Lower kingpin reaction unit vector
#FLKR=lower_kingpin_reaction*LKR

#5 unknowns. 3 dimensional reaction force on the joint. Free to rotate (without getting out of the range provided by the rod-end)
#Plug in previously obtained values 
#Fb=Fb.subs(fb,sol[fb])
#Fc=Fc.subs(fc,sol[fc])

#Set moment arms, moment origin lower kingpin
#W=SA_Lowerarms-Lower_Kingpin
#X=Lower_Fore-Lower_Kingpin
#Z=Lower_Aft-Lower_Kingpin
#
#Moments
#MW=W.cross(Fg)
#MX=X.cross(Fc)
#MZ=Z.cross(Fb)
#
#Eq11=Eq(Fb+Fc+Fg+LKR,Matrix([0,0,0]))
#Eq12=Eq(MW+MX+MZ,Matrix([0,0,0]))
#sol2=solve((Eq11),(fg,lkrx,lkry,lkrz))
#sol2



In [89]:
aft_angle=mat.atan2(d[0],d[1])
aft_angle=np.degrees(aft_angle)
fore_angle=mat.atan2(e[0],e[1])
fore_angle=np.degrees(fore_angle)
aft_angle, fore_angle

(-167.01440376641835, 146.84441420323824)

In [90]:
Fb=Fb.subs(fb,sol[fb])
Fc=Fc.subs(fc,sol[fc])
Fd=Fd.subs(fd,sol[fd])
Fe=Fe.subs(fe,sol[fe])
Ff=Ff.subs(ff,sol[ff])
Fg=Fg.subs(fg,sol[fg])

In [91]:
print(Fb, "Lower AFT")
print(Fc, "Lower Fore")
print(Fd,"Upper Aft")
print(Fe,"Upper Fore")
print(Ff, "Steering")
print(Fg, "SA")

Matrix([[4223.77017195995], [4907.25298160437], [0]]) Lower AFT
Matrix([[6706.46168871336], [-26130.6647505356], [0]]) Lower Fore
Matrix([[2021.41492928495], [8765.76323722010], [730.770257965947]]) Upper Aft
Matrix([[-11831.3447899583], [18110.8152199227], [1509.83374203405]]) Upper Fore
Matrix([[0], [-4532.86468821161], [0]]) Steering
Matrix([[0], [13299.1676403636], [-63516.8246503766]]) SA
